# [Kaggle] 영화 데이터 분석

https://www.kaggle.com/rounakbanik/the-movies-dataset

In [2]:
import pandas as pd
import numpy as np

<br>

## ○ 사용자 코사인 유사도 기반 영화 추천

### 1. 데이터 전처리 & 코사인 유사도 계산

In [5]:
movies = pd.read_csv('data/movies.csv') 
ratings = pd.read_csv('data/ratings_small.csv')
metadata = pd.read_csv('data/movies_metadata.csv')

/opt/anaconda3/envs/mldl/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [6]:
df = pd.merge(ratings, movies, on='movieId')

In [7]:
# 사용자 기준 pivot table
df_matrix_user = df.pivot_table(index='userId', columns='title', values='rating')

# 결측치 0으로 채움
df_matrix_user.fillna(0, inplace=True)
df_matrix_user.head()

title,"""Great Performances"" Cats (1998)",$9.99 (2008),'Hellboy': The Seeds of Creation (2004),'Neath the Arizona Skies (1934),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),...,"Zorro, the Gay Blade (1981)",Zulu (1964),Zulu (2013),[REC] (2007),eXistenZ (1999),loudQUIETloud: A Film About the Pixies (2006),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
# 사용자간 코사인 유사도 계산
from sklearn.metrics.pairwise import cosine_similarity

user_based = cosine_similarity(df_matrix_user)

# 데이터프레임화
user_based = pd.DataFrame(data=user_based, index=df_matrix_user.index, columns=df_matrix_user.index)
user_based.head()

userId,1,2,3,4,5,6,7,8,9,10,...,662,663,664,665,666,667,668,669,670,671
userId,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.000000,0.000000,0.074482,0.016818,0.000000,0.083884,0.000000,0.012843,0.000000,...,0.000000,0.000000,0.014574,0.043719,0.000000,0.000000,0.000000,0.062917,0.000000,0.017466
2,0.000000,1.000000,0.124295,0.118821,0.103646,0.000000,0.212985,0.113190,0.113333,0.043213,...,0.477306,0.116373,0.078284,0.164162,0.466281,0.425462,0.084646,0.024140,0.170595,0.113175
3,0.000000,0.124295,1.000000,0.081640,0.151531,0.060691,0.154714,0.249781,0.134475,0.114672,...,0.161205,0.118207,0.177356,0.158357,0.177098,0.124562,0.124911,0.080984,0.136606,0.170193
4,0.074482,0.118821,0.081640,1.000000,0.130649,0.079648,0.319745,0.191013,0.030417,0.137186,...,0.114319,0.086959,0.137527,0.254030,0.121905,0.088735,0.068483,0.104309,0.054512,0.211609
5,0.016818,0.103646,0.151531,0.130649,1.000000,0.063796,0.095888,0.165712,0.086616,0.032370,...,0.191029,0.038929,0.147187,0.224245,0.139721,0.058252,0.042926,0.038358,0.062642,0.225086


In [9]:
# 영화 기준 pivot table
df_matrix_movie = df.pivot_table(index='title', columns='userId', values='rating')

# 결측치 0으로 채움
df_matrix_movie.fillna(0, inplace=True)

df_matrix_movie.head()

userId,1,2,3,4,5,6,7,8,9,10,...,662,663,664,665,666,667,668,669,670,671
title,,,,,,,,,,,,,,,,,,,,,
"""Great Performances"" Cats (1998)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
$9.99 (2008),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Hellboy': The Seeds of Creation (2004),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Neath the Arizona Skies (1934),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Round Midnight (1986),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
# 영화간의 코사인 유사도 계산
movie_based = cosine_similarity(df_matrix_movie)

movie_based = pd.DataFrame(data=movie_based, index=df_matrix_movie.index, columns=df_matrix_movie.index)
movie_based.head()

title,"""Great Performances"" Cats (1998)",$9.99 (2008),'Hellboy': The Seeds of Creation (2004),'Neath the Arizona Skies (1934),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),...,"Zorro, the Gay Blade (1981)",Zulu (1964),Zulu (2013),[REC] (2007),eXistenZ (1999),loudQUIETloud: A Film About the Pixies (2006),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
title,,,,,,,,,,,,,,,,,,,,,
"""Great Performances"" Cats (1998)",1.000000,0.0,0.0,0.164399,0.020391,0.0,0.014046,0.000000,0.0,0.003166,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0
$9.99 (2008),0.000000,1.0,0.0,0.000000,0.000000,0.0,0.000000,0.079474,0.0,0.156330,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.013899,0.0,0.058218,0.0
'Hellboy': The Seeds of Creation (2004),0.000000,0.0,1.0,0.000000,0.000000,1.0,0.000000,0.217357,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0
'Neath the Arizona Skies (1934),0.164399,0.0,0.0,1.000000,0.124035,0.0,0.085436,0.000000,0.0,0.019259,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0
'Round Midnight (1986),0.020391,0.0,0.0,0.124035,1.000000,0.0,0.010597,0.143786,0.0,0.136163,...,0.0,0.0,0.0,0.0,0.121567,0.0,0.000000,0.0,0.000000,0.0


### 2. 영화 추천 서비스 구현

In [11]:
# 비슷한 평점을 매긴(비슷한 취향을 가진) 사용자 상위 n명
def get_similar_users(user, n):
    return user_based[user].sort_values(ascending=False)[:n]

get_similar_users(18, 4) # 자기 자신 포함

userId
18     1.000000
70     0.660646
284    0.643825
256    0.640314
Name: 18, dtype: float64

In [12]:
# 별점 비교와 정렬을 위해 reset index
df_matrix_movie_2 = df_matrix_movie.reset_index()

# 예) 사용자1의 평점 높은 영화 5개
df_matrix_movie_2[['title', 1]].sort_values(1, ascending=False).head() 

userId,title,1
1593,Cinema Paradiso (Nuovo cinema Paradiso) (1989),4.0
8056,Tron (1982),4.0
2926,"French Connection, The (1971)",4.0
2275,Dracula (Bram Stoker's Dracula) (1992),3.5
7090,Sleepers (1996),3.0


In [13]:
# 추천을 원하는 사용자가 이미 본 영화 제거
def del_watched_movie(user, n):
  # user: 추천을 원하는 사용자 / n: 유사도가 높은 사용자
  return df_matrix_movie_2[df_matrix_movie_2[user]==0][['title', user, n]]

In [14]:
def recommend_movie_by_user(user):
    
    # 자기 자신 제외 유사도 높은 사용자 리스트 (3명)
    users = get_similar_users(user, 4).index[1:4] 
    
    rcmd_movies = []
    for i in users:
        # 이미 본 영화 제외 유사도가 높은 사용자가 높게 별점을 준 영화 리스트 받아옴
        movie_list = list(del_watched_movie(user, i).sort_values(i, ascending=False).title[:5])
        rcmd_movies.append(movie_list)

    return rcmd_movies

In [15]:
recommend_movie_by_user(3)

[['Green Mile, The (1999)',
  'Pianist, The (2002)',
  'Donnie Darko (2001)',
  'Big Lebowski, The (1998)',
  'Lives of Others, The (Das leben der Anderen) (2006)'],
 ['Lion King, The (1994)',
  'Toy Story (1995)',
  'Star Wars: Episode IV - A New Hope (1977)',
  'Shrek (2001)',
  'Incredibles, The (2004)'],
 ['Crimson Tide (1995)',
  'Rob Roy (1995)',
  'Jurassic Park (1993)',
  'Clear and Present Danger (1994)',
  'Outbreak (1995)']]

In [16]:
user_num = int(input('추천을 원하는 사용자 번호를 입력하세요: '))
recommend_movie_by_user(user_num)

추천을 원하는 사용자 번호를 입력하세요: 10


[['Lord of the Rings: The Return of the King, The (2003)',
  'Gladiator (2000)',
  'Silence of the Lambs, The (1991)',
  'Spider-Man 2 (2004)',
  'Star Wars: Episode IV - A New Hope (1977)'],
 ['Star Wars: Episode IV - A New Hope (1977)',
  'Wallace & Gromit: The Wrong Trousers (1993)',
  "One Flew Over the Cuckoo's Nest (1975)",
  'Men in Black (a.k.a. MIB) (1997)',
  'Blade Runner (1982)'],
 ['Stripes (1981)',
  'Breakfast Club, The (1985)',
  'On Golden Pond (1981)',
  'Rain Man (1988)',
  'Back to the Future (1985)']]

<br>

### ++) 영화간의 코사인 유사도를 이용한 영화 추천

In [17]:
def recommend_movie_by_rating(title):
    return movie_based[title].sort_values(ascending=False)[:5]

In [18]:
recommend_movie_by_rating('Godfather, The (1972)')

title
Godfather, The (1972)                     1.000000
Godfather: Part II, The (1974)            0.773685
Goodfellas (1990)                         0.620349
One Flew Over the Cuckoo's Nest (1975)    0.568244
American Beauty (1999)                    0.557997
Name: Godfather, The (1972), dtype: float64

<br>

## ○ 새로운 영화에 대한 별점 예측 모델

### 1. 데이터 전처리
#### 1-1. movies_metadata 

In [15]:
metadata.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [16]:
metadata_origin = metadata.copy()

# 청불 여부, 장르, 제목, 예산, 언어, 줄거리, 유명도, 제작사, 제작 국가, 상영시간, 평균 별점, 별점 수 등
meta_columns = ['id', 'title', 'adult', 'budget', 'original_language', 'popularity', 'production_companies', 'production_countries', 'runtime','vote_average', 'vote_count']

metadata = metadata[meta_columns]
metadata.head()

,id,title,adult,budget,original_language,popularity,production_companies,production_countries,runtime,vote_average,vote_count
0,862,Toy Story,False,30000000,en,21.9469,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",81.0,7.7,5415.0
1,8844,Jumanji,False,65000000,en,17.0155,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",104.0,6.9,2413.0
2,15602,Grumpier Old Men,False,0,en,11.7129,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",101.0,6.5,92.0
3,31357,Waiting to Exhale,False,16000000,en,3.85949,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",127.0,6.1,34.0
4,11862,Father of the Bride Part II,False,0,en,8.38752,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",106.0,5.7,173.0


In [17]:
# production_companies

def get_production_com(s):
    try:
        result = eval(s)[0]['name']
    except:
        return -1 # [] 예외처리
    return result

metadata['production_companies'] = metadata['production_companies'].apply(get_production_com)

In [18]:
# production_countries

def get_production_country(s):
    try:
        result = eval(s)[0]['iso_3166_1']
    except:
        return -1 # [] 예외처리
    return result

metadata['production_countries'] = metadata['production_countries'].apply(get_production_country)

In [19]:
# production_companies, production_countries 예외처리된 행 확인
metadata[(metadata.production_companies == -1) | (metadata.production_countries == -1)].head()

,id,title,adult,budget,original_language,popularity,production_companies,production_countries,runtime,vote_average,vote_count
50,117164,Guardian Angel,False,0,en,0.595949,-1,-1,93.0,6.3,3.0
52,49133,Lamerica,False,0,it,1.36129,-1,IT,116.0,7.7,11.0
55,124057,Kids of the Round Table,False,0,en,0.307075,Telefilm Canada,-1,89.0,3.0,1.0
57,11010,The Postman,False,0,it,10.1684,-1,BE,108.0,7.6,181.0
58,99040,The Confessional,False,0,fr,0.120789,-1,CA,100.0,6.5,2.0


In [20]:
metadata = metadata[(metadata.production_countries != -1) & (metadata.production_countries != -1)]
metadata.head()

,id,title,adult,budget,original_language,popularity,production_companies,production_countries,runtime,vote_average,vote_count
0,862,Toy Story,False,30000000,en,21.9469,Pixar Animation Studios,US,81.0,7.7,5415.0
1,8844,Jumanji,False,65000000,en,17.0155,TriStar Pictures,US,104.0,6.9,2413.0
2,15602,Grumpier Old Men,False,0,en,11.7129,Warner Bros.,US,101.0,6.5,92.0
3,31357,Waiting to Exhale,False,16000000,en,3.85949,Twentieth Century Fox Film Corporation,US,127.0,6.1,34.0
4,11862,Father of the Bride Part II,False,0,en,8.38752,Sandollar Productions,US,106.0,5.7,173.0


<br>

#### 1-2. links & movies_metadata

In [21]:
links = pd.read_csv('data/links.csv')
links.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [22]:
metadata.head()

,id,title,adult,budget,original_language,popularity,production_companies,production_countries,runtime,vote_average,vote_count
0,862,Toy Story,False,30000000,en,21.9469,Pixar Animation Studios,US,81.0,7.7,5415.0
1,8844,Jumanji,False,65000000,en,17.0155,TriStar Pictures,US,104.0,6.9,2413.0
2,15602,Grumpier Old Men,False,0,en,11.7129,Warner Bros.,US,101.0,6.5,92.0
3,31357,Waiting to Exhale,False,16000000,en,3.85949,Twentieth Century Fox Film Corporation,US,127.0,6.1,34.0
4,11862,Father of the Bride Part II,False,0,en,8.38752,Sandollar Productions,US,106.0,5.7,173.0


In [23]:
metadata.id = metadata.id.apply(lambda x: int(x)) # id -> int

# metadata 기준 merge
links_metadata = pd.merge(metadata[meta_columns], links, left_on = 'id', right_on='tmdbId', how='left')
del links_metadata['tmdbId'] # id = tmdbId

# reset index
links_metadata.reset_index(drop=True)
links_metadata.head()

,id,title,adult,budget,original_language,popularity,production_companies,production_countries,runtime,vote_average,vote_count,movieId,imdbId
0,862,Toy Story,False,30000000,en,21.9469,Pixar Animation Studios,US,81.0,7.7,5415.0,1,114709
1,8844,Jumanji,False,65000000,en,17.0155,TriStar Pictures,US,104.0,6.9,2413.0,2,113497
2,15602,Grumpier Old Men,False,0,en,11.7129,Warner Bros.,US,101.0,6.5,92.0,3,113228
3,31357,Waiting to Exhale,False,16000000,en,3.85949,Twentieth Century Fox Film Corporation,US,127.0,6.1,34.0,4,114885
4,11862,Father of the Bride Part II,False,0,en,8.38752,Sandollar Productions,US,106.0,5.7,173.0,5,113041


<br>

#### 1-3. ratings

In [24]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [25]:
# 사용자 리스트
user_list = list(set(ratings['userId']))

# 사용자별 별점 수 리스트
counts = []
for id in user_list:
    counts.append(len(ratings[ratings.userId == id]))
    
rating_counts = pd.DataFrame({'userId':user_list, 'counts':counts})
rating_counts[rating_counts.counts == max(counts)]

,userId,counts
546,547,2391


In [26]:
# 별점을 가장 많이 준 사용자 547의 별점 데이터만 선택
rating_usr = ratings[ratings.userId == 547][['movieId', 'rating']]
rating_usr.head()

,movieId,rating
78433,1,3.5
78434,6,2.5
78435,7,2.0
78436,11,3.0
78437,14,3.5


<br>

#### 1-4. ratings & links_metadata

In [27]:
rating_usr.head()

,movieId,rating
78433,1,3.5
78434,6,2.5
78435,7,2.0
78436,11,3.0
78437,14,3.5


In [28]:
links_metadata.head()

,id,title,adult,budget,original_language,popularity,production_companies,production_countries,runtime,vote_average,vote_count,movieId,imdbId
0,862,Toy Story,False,30000000,en,21.9469,Pixar Animation Studios,US,81.0,7.7,5415.0,1,114709
1,8844,Jumanji,False,65000000,en,17.0155,TriStar Pictures,US,104.0,6.9,2413.0,2,113497
2,15602,Grumpier Old Men,False,0,en,11.7129,Warner Bros.,US,101.0,6.5,92.0,3,113228
3,31357,Waiting to Exhale,False,16000000,en,3.85949,Twentieth Century Fox Film Corporation,US,127.0,6.1,34.0,4,114885
4,11862,Father of the Bride Part II,False,0,en,8.38752,Sandollar Productions,US,106.0,5.7,173.0,5,113041


In [29]:
df = pd.merge(rating_usr, links_metadata, on='movieId')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2325 entries, 0 to 2324
Data columns (total 14 columns):
movieId                 2325 non-null int64
rating                  2325 non-null float64
id                      2325 non-null int64
title                   2325 non-null object
adult                   2325 non-null object
budget                  2325 non-null object
original_language       2325 non-null object
popularity              2325 non-null object
production_companies    2325 non-null object
production_countries    2325 non-null object
runtime                 2325 non-null float64
vote_average            2325 non-null float64
vote_count              2325 non-null float64
imdbId                  2325 non-null int64
dtypes: float64(4), int64(3), object(7)
memory usage: 272.5+ KB


<br>

### 2. 모델링 데이터 전처리

#### 2-1. X 데이터

In [30]:
df.head()

,movieId,rating,id,title,adult,budget,original_language,popularity,production_companies,production_countries,runtime,vote_average,vote_count,imdbId
0,1,3.5,862,Toy Story,False,30000000,en,21.9469,Pixar Animation Studios,US,81.0,7.7,5415.0,114709
1,6,2.5,949,Heat,False,60000000,en,17.9249,Regency Enterprises,US,170.0,7.7,1886.0,113277
2,7,2.0,11860,Sabrina,False,58000000,en,6.67728,Paramount Pictures,DE,127.0,6.2,141.0,114319
3,11,3.0,9087,The American President,False,62000000,en,6.31844,Columbia Pictures,US,106.0,6.5,199.0,112346
4,14,3.5,10858,Nixon,False,44000000,en,5.092,Hollywood Pictures,US,192.0,7.1,72.0,113987


In [31]:
X_data = df.copy()

x_columns = ['rating', 'adult', 'budget', 'original_language', 'popularity', 'runtime', 'vote_average', 'vote_count']
X_data = X_data[x_columns]

del X_data['adult'] # 모든 행의 adult == False

# original language -> one hot vector
lang_df = pd.get_dummies(X_data['original_language'], prefix='lang')
X_data = X_data.join(lang_df)
del X_data['original_language']

X_data.head()

,rating,budget,popularity,runtime,vote_average,vote_count,lang_af,lang_cn,lang_da,lang_de,...,lang_he,lang_hi,lang_it,lang_ja,lang_nl,lang_pl,lang_pt,lang_sv,lang_th,lang_zh
0,3.5,30000000,21.9469,81.0,7.7,5415.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2.5,60000000,17.9249,170.0,7.7,1886.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2.0,58000000,6.67728,127.0,6.2,141.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3.0,62000000,6.31844,106.0,6.5,199.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,3.5,44000000,5.092,192.0,7.1,72.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


<br>

#### 2-2. X, y 데이터

In [32]:
y_data = X_data[['rating']]
y_data.head()

,rating
0,3.5
1,2.5
2,2.0
3,3.0
4,3.5


In [33]:
del X_data['rating']
X_data.head()

,budget,popularity,runtime,vote_average,vote_count,lang_af,lang_cn,lang_da,lang_de,lang_en,...,lang_he,lang_hi,lang_it,lang_ja,lang_nl,lang_pl,lang_pt,lang_sv,lang_th,lang_zh
0,30000000,21.9469,81.0,7.7,5415.0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,60000000,17.9249,170.0,7.7,1886.0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,58000000,6.67728,127.0,6.2,141.0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,62000000,6.31844,106.0,6.5,199.0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,44000000,5.092,192.0,7.1,72.0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


<br>

#### 2-3. train / test data split

In [34]:
X_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2325 entries, 0 to 2324
Data columns (total 22 columns):
budget          2325 non-null object
popularity      2325 non-null object
runtime         2325 non-null float64
vote_average    2325 non-null float64
vote_count      2325 non-null float64
lang_af         2325 non-null uint8
lang_cn         2325 non-null uint8
lang_da         2325 non-null uint8
lang_de         2325 non-null uint8
lang_en         2325 non-null uint8
lang_es         2325 non-null uint8
lang_fr         2325 non-null uint8
lang_he         2325 non-null uint8
lang_hi         2325 non-null uint8
lang_it         2325 non-null uint8
lang_ja         2325 non-null uint8
lang_nl         2325 non-null uint8
lang_pl         2325 non-null uint8
lang_pt         2325 non-null uint8
lang_sv         2325 non-null uint8
lang_th         2325 non-null uint8
lang_zh         2325 non-null uint8
dtypes: float64(3), object(2), uint8(17)
memory usage: 227.6+ KB


In [35]:
# object -> float
X_data.budget = X_data.budget.apply(lambda x: float(x))
X_data.popularity = X_data.popularity.apply(lambda x: float(x))

y_data2 = y_data.copy()
y_data2.rating = y_data2.rating.apply(lambda x: int(x*10))

In [36]:
# train / test
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X_data, y_data2, test_size=0.3, random_state=0)

<br>

### 3. XGBoost

In [37]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

In [38]:
# create model
model = XGBClassifier(seed = 0, n_jobs = -1, learning_rate = 0.1, n_estimators = 100, max_depth = 3) 

In [39]:
# fit
model.fit(x_train, y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=-1, nthread=None, objective='multi:softprob', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=0, silent=True,
       subsample=1)

In [40]:
y_pred = model.predict(x_test)
accuracy_score(y_test, y_pred)

0.26504297994269344

<br>

### 4.  RandomForest

In [41]:
from sklearn.ensemble import RandomForestClassifier

In [55]:
# create model
clf = RandomForestClassifier(n_estimators=13)

In [56]:
# train / test
x_train, x_test, y_train, y_test = train_test_split(X_data, y_data2, test_size=0.3, random_state=10)

In [57]:
clf.fit(x_train, y_train)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=13, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [58]:
y_pred = clf.predict(x_test)
clf.score(x_test, y_test)

0.22492836676217765

<br>

#### ++) Hyperparameter Optimization

In [59]:
from sklearn.model_selection import GridSearchCV

xgb = XGBClassifier()

param_grid = {'max_depth': [3, 5, 7],
              'learning_rate': [0.01, 0.1, 0.2, 0,3],
              'subsample': [0.6, 0.8, 1.0]}

# param_grid = {
#         'silent': [False],
#         'max_depth': [3, 5, 7], 
#         'learning_rate': [0.01, 0.1, 0.2, 0,3],
#         'min_child_weight': [1.0, 3.0, 5.0, 7.0, 10.0],
#         'n_estimators': [100]}

grid = GridSearchCV(xgb, param_grid, refit=True, verbose=1)

grid.fit(x_train, y_train)
print('The best parameters are ', grid.best_params_)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Fitting 3 folds for each of 45 candidates, totalling 135 fits


[Parallel(n_jobs=1)]: Done 135 out of 135 | elapsed:  4.0min finished


The best parameters are  {'learning_rate': 0.01, 'max_depth': 3, 'subsample': 0.6}


In [60]:
params = {'learning_rate': 0.01, 'max_depth': 3, 'subsample': 0.6}

xgb = XGBClassifier(**params)
xgb.fit(x_train, y_train)
accuracy_score(y_test, xgb.predict(x_test))

0.2707736389684814